# Chapter 4: CLOCK ARITHMETIC

Calling modulo arithmetic "clock arithmetic" is a way of calling attention to the finite cyclic nature of always taking remainders, factoring away the "modulus".  What does that mean?

In Python, one of our primitive operators is %, for "modulo", which relates to the built-in function divmod(&nbsp;).  Lets see how % and divmod do their jobs:

In [1]:
(5 + 5) % 10 # no remainder

0

In [2]:
divmod(10, 5) # 5 goes into 10 twice, no remainder

(2, 0)

In [3]:
19 % 10 # divide by 10, give the remainder

9

The unit circle of 360 degrees, or 2 pi radians, is also treated in a clock-like fashion.  Going around thousands of degrees from some present position, never takes us beyond 360.  We're confined to a finite domain.  

There's a kind of "closure" in this picture, which might be resonating with you by now, as one of those properties of a group.  Will we find groups in this Chapter?  You bet.

We call this "clock arithmetic" because even if we say "20 hours from now", or "30 hours ago" we'll still be somewhere on the circle marked out into 12 intervals, each representing one hour.  The yearly calendar is likewise a kind of modulus.  No matter how many days we add, we're still somewhere between 0 and 365 days into some year.

### A Class for M-numbers
Lets build a class, the instances of which will multiply and add per some fixed modulus, meaning we're always factoring out the modulus and keeping the remainder.  

In [4]:
class M:  # for "modulo"
    
    modulus = 10  # class level
    
    def __init__(self, val):
        self.val = val % M.modulus
        
    def __add__(self, other):
        return M((self.val + other.val) % M.modulus)
    
    def __mul__(self, other):
        return M((self.val * other.val) % M.modulus)
    
    def __pow__(self, exp):
        raise NotImplemented
    
    def __repr__(self):
        return "(" + str(self.val) + ")"
    
a = M(8)
b = M(7)
print(a, b)
print("(8) * (7) = ", a * b)
print("(8) + (7) = ", a + b)

(8) (7)
(8) * (7) =  (6)
(8) + (7) =  (5)


OK, everything seems to be working, even though we haven't implemented powering yet.  Eventually we'd like to go pow(M(3), -1) to get the inverse of M(3), such that M(3) times its inverse equals the multiplicative identity M(1).

### Looking for Groups

But wait, does every M-number, with modulus set to 10, have an inverse?  We can check that easily.  First, lets make a list of all 10 M-numbers, (0) through (9):

In [5]:
elems = [M(n) for n in range(10)]
elems

[(0), (1), (2), (3), (4), (5), (6), (7), (8), (9)]

Now we can do like a "times table" wherein we pick a single M-number, say M(5), and multiply it by every number in elems...

In [6]:
[M(5) * x for x in elems] 

[(0), (5), (0), (5), (0), (5), (0), (5), (0), (5)]

Interesting.  In ordinary arithematic, the times table for 5 goes 0, 5, 10, 15, 20, 25, 30... and so on.  Factoring out the 10s, leaving only remainders, we get (0) or (5) as our two M-numbers.  We have no way to reach (1) and so (5) has no inverse.  We don't have a group yet.

Lets try M(2):

In [7]:
[M(2) * x for x in elems]

[(0), (2), (4), (6), (8), (0), (2), (4), (6), (8)]

Same thing.  We cycle around and around, always stopping at the same stations, like a model train going in a circle.  We never stop at (1).  So M(2) has no inverse either.  What about M(3)?

In [8]:
[M(3) * x for x in elems]

[(0), (3), (6), (9), (2), (5), (8), (1), (4), (7)]

Aha!  Now we're getting somewhere.  M(3) * M(7) returns M(1), so these two are inverses of one another.  One fact to notice immediately is neither has any factors in common with 10. In fact, both are prime numbers in the ordinary integer sense.  M(9) is not prime, but again, 9 has no factors in common with 10.  So does M(9) have an inverse?  Lets check:

In [9]:
[M(9) * x for x in elems]

[(0), (9), (8), (7), (6), (5), (4), (3), (2), (1)]

Indeed it does.  M(9) * M(9) = M(1), meaning M(9) is its own inverse.

When positive integers have no factors in common, aside from 1, we say they're "relatively prime" or that they're "strangers".  3 and 10 are strangers, as are 9 and 10.  Sometimes we write 7 | 10 = 1, meaning their greatest factor in common is 1.  On the other hand, 6 | 10 = 2, as 2 divides into both without remainder.


### Finding Totatives

It'd sure be handy at this point, to have a function, gcd, that returns the greatest common divisor of two numbers.  Then we could find all the strangers to 10, that are positive integers less than 10, pretty easily.  We call these strangers the "totatives" of 10, and the number of totatives, is called the "totient" of 10.

In [10]:
import math
modulus = 10
[x for x in range(modulus) if math.gcd(x, modulus) == 1]

[1, 3, 7, 9]

In [11]:
modulus = 12
[x for x in range(modulus) if math.gcd(x, modulus) == 1]

[1, 5, 7, 11]

That was pretty easy.  Using Python's "list comprehension" syntax, which allows an if clause for filtering, we were able to get the totatives of 10 and 12 respectively.  Both 10 and 12 have a totient of four, meaning each has four totatives.

What's true is that the M-numbers that are totatives of some modulus (say 10), collectively form a group under multiplication.  If we confine ourselves to strangers to the modulus, we'll have an inverse for every element, closure, associativity, and M(1) will be included.  That's a quick way to get a group.

Lets build a function for computing totatives and use that to build some groups.

In [12]:
def totatives(n):
    return {x for x in range(n) if math.gcd(x, n) == 1}

In [13]:
M.modulus = 12
elems = {M(x) for x in totatives(12)}
elems

{(11), (1), (7), (5)}

In [14]:
M.modulus = 100
elems = {M(x) for x in totatives(100)}
elems

{(37),
 (39),
 (41),
 (43),
 (47),
 (49),
 (51),
 (53),
 (9),
 (57),
 (59),
 (61),
 (63),
 (67),
 (69),
 (71),
 (73),
 (77),
 (79),
 (1),
 (81),
 (83),
 (87),
 (27),
 (89),
 (29),
 (91),
 (3),
 (93),
 (13),
 (97),
 (23),
 (99),
 (19),
 (11),
 (7),
 (21),
 (31),
 (17),
 (33)}

If we confine ourselves to a set of strangers, we're safe in assuming there's always an inverse for any given element.  That suggests a "brute force" way of finding any element's inverse:  just multiply it by every element in the same set of strangers, until their product is 1 (the identity element).  The function below accomplishes this:

In [15]:
def inverse(n : M, elems : set) -> M:
    for m in elems:
        if (n * m).val == 1:
            return m

In [16]:
inverse(M(79), elems)

(19)

In [17]:
M(19) * M(79)

(1)

### Adding a Power Method

We now have the technology (tool set) we need to add the \_\_pow\_\_ method to our M class. A negative powering triggers finding an inverse of self, then positively powering.  We're saying M(n) \*\* -x equals (M(n)\*\*-1)\*\*x.

This is how we treat exponents normally, e.g. 2 \*\* -2 equals (2 \*\* -1)\*\*2 or 1/4.

For example, pow(M(79), -2) means find the inverse of M(79) e.g. M(79)\*\*-1, and then raise the result to the 2nd power.

In [18]:
class M:  # for "modulo"
    
    modulus = 10  # class level
    
    def __init__(self, val):
        self.val = val % M.modulus
        
    def __add__(self, other):
        return M((self.val + other.val) % M.modulus)
    
    def __mul__(self, other):
        return M((self.val * other.val) % M.modulus)
    
    def __pow__(self, exp):  # pow() and ** both trigger this method
        output = self
        if exp < 0:
            exp = abs(exp)
            output = inverse(self, {M(x) for x in totatives(M.modulus)})
        elif exp == 0:
            output = M(1)
        elif exp == 1:
            output = self    
        if exp > 1:
            for _ in range(1, exp):
                output *= self
        
        return output
    
    def __repr__(self):
        return "(" + str(self.val) + ")"

In [19]:
M.modulus = 100
a = M(79)
a_inv = pow(a, -1)
a_inv

(19)

In [20]:
M(13) * M(13) * M(13)

(97)

In [21]:
M(13)**3       # confirm this is another way of saying it (we're testing)

(97)

In [22]:
M(13)**-1      # give me the inverse of M(13) please

(77)

In [23]:
M(13) * M(77)  # these must be inverses then

(1)

So now it looks like we have another fun tool for exploring Group Theory, while learning Python at the same time!

Back to Chapter 3: [A First Class](A%20First%20Class.ipynb)